In [ ]:
# base_folder = "/content/drive/MyDrive/Colab Notebooks/housing_fall2025"
# %cd "{base_folder}"

In [2]:
from pathlib import Path
import sqlite3
import pandas as pd
BASE_DIR = Path("..")          # project root
DATA_DIR = BASE_DIR / "data"
MODELS_DIR = BASE_DIR / "models"
conn = sqlite3.connect(BASE_DIR / "notebooks" / "housing.db")


housing = pd.read_sql_query(
    """
    SELECT
        b.block_id,
        b.longitude,
        b.latitude,
        s.housing_median_age,
        s.total_rooms,
        s.total_bedrooms,
        s.population,
        s.households,
        s.median_income,
        s.median_house_value,
        op.name AS ocean_proximity
    FROM block AS b
    JOIN block_housing_stats AS s
        ON s.block_id = b.block_id
    JOIN ocean_proximity AS op
        ON op.ocean_proximity_id = b.ocean_proximity_id
    ORDER BY b.block_id
    """,
    conn,
)
conn.close()

housing.head()

,block_id,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,0,-122.23,37.88,41.0,880,129.0,322,126,8.3252,452600.0,NEAR BAY
1,1,-122.22,37.86,21.0,7099,1106.0,2401,1138,8.3014,358500.0,NEAR BAY
2,2,-122.24,37.85,52.0,1467,190.0,496,177,7.2574,352100.0,NEAR BAY
3,3,-122.25,37.85,52.0,1274,235.0,558,219,5.6431,341300.0,NEAR BAY
4,4,-122.25,37.85,52.0,1627,280.0,565,259,3.8462,342200.0,NEAR BAY


In [3]:
# =============================================================================
# ANALYZE HOUSING DATA FOR STREAMLIT APP
# Find min/max/median for numerical features and unique values for categorical features
# =============================================================================

import json

print("=" * 80)
print("ANALYZING HOUSING DATA FOR STREAMLIT APP")
print("=" * 80)

# Define the features we need for prediction
numerical_features = [
    'longitude',
    'latitude',
    'housing_median_age',
    'total_rooms',
    'total_bedrooms',
    'population',
    'households',
    'median_income'
]

categorical_features = ['ocean_proximity']

# Create schema dictionary
data_schema = {
    "numerical": {},
    "categorical": {}
}

# Analyze numerical features
print("\n" + "-" * 80)
print("NUMERICAL FEATURES")
print("-" * 80)
print(f"{'Feature':<25} {'Min':<15} {'Max':<15} {'Mean':<15} {'Median':<15}")
print("-" * 80)

for feature in numerical_features:
    min_val = float(housing[feature].min())
    max_val = float(housing[feature].max())
    mean_val = float(housing[feature].mean())
    median_val = float(housing[feature].median())

    data_schema["numerical"][feature] = {
        "min": min_val,
        "max": max_val,
        "mean": mean_val,
        "median": median_val
    }

    print(f"{feature:<25} {min_val:<15.2f} {max_val:<15.2f} {mean_val:<15.2f} {median_val:<15.2f}")

# Analyze categorical features
print("\n" + "-" * 80)
print("CATEGORICAL FEATURES")
print("-" * 80)

for feature in categorical_features:
    unique_values = housing[feature].unique().tolist()
    value_counts = housing[feature].value_counts().to_dict()

    data_schema["categorical"][feature] = {
        "unique_values": unique_values,
        "value_counts": value_counts
    }

    print(f"\n{feature}:")
    print(f"  Unique values: {unique_values}")
    print(f"  Value counts:")
    for value, count in value_counts.items():
        print(f"    {value}: {count} ({count/len(housing)*100:.1f}%)")

# Save schema to JSON file
output_file = DATA_DIR / "data_schema.json"

with open(output_file, "w") as f:
    json.dump(data_schema, f, indent=2)


print("\n" + "=" * 80)
print(f"✓ Data schema saved to {output_file}")
print("=" * 80)

# Display the JSON structure
print("\n" + "-" * 80)
print("GENERATED SCHEMA (data_schema.json)")
print("-" * 80)
print(json.dumps(data_schema, indent=2))

print("\n" + "=" * 80)
print("DONE! Use data_schema.json in your Streamlit app")
print("=" * 80)

ANALYZING HOUSING DATA FOR STREAMLIT APP

--------------------------------------------------------------------------------
NUMERICAL FEATURES
--------------------------------------------------------------------------------
Feature                   Min             Max             Mean            Median         
--------------------------------------------------------------------------------
longitude                 -124.35         -114.31         -119.57         -118.49        
latitude                  32.54           41.95           35.63           34.26          
housing_median_age        1.00            52.00           28.64           29.00          
total_rooms               2.00            39320.00        2635.76         2127.00        
total_bedrooms            1.00            6445.00         537.87          435.00         
population                3.00            35682.00        1425.48         1166.00        
households                1.00            6082.00         499.54  